In [77]:
import pandas as pd
import numpy as np

import requests

from datetime import datetime, timedelta
from pprint import pprint

import plotly.express as px
import plotly.colors as pc

### EJEMPLO


In [78]:
lang    =  "es"
category = "balance"
widget   = "balance-electrico"
query   = "start_date=2018-01-01T00:00&end_date=2018-12-31T23:59&time_trunc=year"


In [79]:
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Host' : 'apidatos.ree.es'
}

In [80]:
endpoint = f"https://apidatos.ree.es/{lang}/datos/{category}/{widget}?{query}"
response = requests.get(url = endpoint, headers = headers)
data = response.json()
pprint(data)

{'data': {'attributes': {'description': 'Balance eléctrico: asignación de '
                                        'unidades de producción según '
                                        'combustible principal. La producción '
                                        'neta de las instalaciones no '
                                        'renovables e hidráulicas UGH tiene '
                                        'descontados los consumos propios de '
                                        'las instalaciones. En dichos tipos de '
                                        'producción una generación negativa '
                                        'indica que la electricidad horaria '
                                        'consumida para los usos de las '
                                        'plantas excede la producción horaria '
                                        'de las instalaciones.',
                         'last-update': '2019-06-12T17:40:40.000+02:00',
              

# OBTENCIÓN DE DATOS DE BALANCE




In [81]:
lang    =  input('¿En español (es) o en inglés (en)?') 

restaDia = float(input("¿Cuantos días atrás?: ______________"))
input_año = int(input("¿Qué año?: ______________"))

In [82]:
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Host' : 'apidatos.ree.es'
}

In [83]:
def balance_datos(lang, category, widget):

    now = datetime.now()
    ultima_fecha = (now - timedelta(days = restaDia)).replace(year = input_año).strftime('%Y-%m-%d')


    hoy = now.replace(year = input_año).strftime('%Y-%m-%d')

    query = f"start_date={ultima_fecha}T00:00&end_date={hoy}T23:59&time_trunc=day"



    endpoint = f"https://apidatos.ree.es/{lang}/datos/balance/balance-electrico?{query}"
    response = requests.get(url = endpoint, headers = headers)
    data = response.json()



    lista_nombres = list()
    lista_tipos = list()
    lista_valores = list()
    lista_porcentajes = list()
    lista_fechas = list()
    lista_dias = list()
    lista_meses = list()
    lista_años = list()


    for dato in data['included']: 
        for info in dato['attributes']['content']:
            nombre = info['type']

            tipo = info['groupId']


            for i in info['attributes']['values']:
                valor = i['value']

                porcentaje = i['percentage']

                fecha = i['datetime']
                fecha = pd.to_datetime(fecha)
                dia = fecha.strftime('%d')
                mes = fecha.strftime('%m')
                año = fecha.strftime('%Y')

                fecha = fecha.strftime("%d/%m/%Y")

        
            
            
                lista_nombres.append(nombre)
                lista_tipos.append(tipo)
                lista_valores.append(valor)
                lista_porcentajes.append(porcentaje)
                lista_fechas.append(fecha)
                lista_dias.append(dia)
                lista_meses.append(mes)
                lista_años.append(año)

    df_balance = pd.DataFrame()
    df_balance['nombre']               = lista_nombres
    df_balance['tipo de energía']      = lista_tipos
    df_balance['Valores']              = lista_valores
    df_balance["Porcentaje"]           = lista_porcentajes
    df_balance["Fecha actualización"]  = lista_fechas
               




    return df_balance

df_balance  = balance_datos(lang, category, widget)

df_balance


,nombre,tipo de energía,Valores,Porcentaje,Fecha actualización
0,Hidráulica,Renovable,23389.362,0.064915,30/09/2022
1,Hidráulica,Renovable,18682.792,0.081283,01/10/2022
2,Hidráulica,Renovable,15426.459,0.067843,02/10/2022
3,Hidráulica,Renovable,28347.218,0.174671,03/10/2022
4,Hidráulica,Renovable,23889.363,0.131823,04/10/2022
...,...,...,...,...,...
226,Demanda en b.c.,Demanda en b.c.,675127.531,1.000000,06/10/2022
227,Demanda en b.c.,Demanda en b.c.,666351.834,1.000000,07/10/2022
228,Demanda en b.c.,Demanda en b.c.,589542.598,1.000000,08/10/2022
229,Demanda en b.c.,Demanda en b.c.,540796.450,1.000000,09/10/2022


In [84]:
df_balance.to_csv('balance_electrico.csv')

In [85]:
colores_personalizados = px.colors.qualitative.Plotly + px.colors.qualitative.Pastel + px.colors.qualitative.Set1
colores_personalizados = colores_personalizados[:30]

In [86]:
fig1 = px.strip(data_frame = df_balance,
         x          = "Fecha actualización",
         y          = "Valores",
         color      = "nombre",
         color_discrete_sequence = colores_personalizados,
         hover_data = df_balance.columns)

fig1.show()


/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [87]:
fig2 = px.strip(data_frame = df_balance,
         x          = "Fecha actualización",
         y          = "Porcentaje",
         color      = "tipo de energía",
         color_discrete_sequence = colores_personalizados,
         hover_data = df_balance.columns)

fig2.show()

/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [111]:
px.line(data_frame = df_balance,
        x = 'Fecha actualización',
        y = 'Valores',
        color = 'nombre',
        color_discrete_sequence = colores_personalizados,
)

#Dividir por energías (tres gráficas)


/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [98]:
px.scatter(data_frame = df_balance,
            y = 'Valores',
            color = 'nombre',
            color_discrete_sequence = colores_personalizados,
            opacity     = 0.5,
            size        = 'Porcentaje',
            size_max    = 30
          )

/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [121]:
px.scatter(data_frame = df_balance,
            y = 'Valores',
            color = 'tipo de energía',
            color_discrete_sequence = colores_personalizados,
            opacity     = 0.5,
            hover_name  = 'nombre',
            size        = 'Porcentaje',
            size_max    = 30
          )

/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [69]:
px.box(data_frame=df_balance,
       x = 'Valores',
       y = 'nombre',
       color = 'nombre',
       color_discrete_sequence = colores_personalizados)

/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [71]:
px.box(data_frame=df_balance,
       x = 'Valores',
       y = 'tipo de energía',
       color = 'tipo de energía',
       color_discrete_sequence = colores_personalizados)

/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

